In [2]:
import pandas as pd
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az
from statsmodels.tsa.stattools import adfuller
from datetime import datetime
from flask import Flask, jsonify
import os

In [3]:
# Load and preprocess Brent oil price data
df = pd.read_csv('../data/BrentOilPrices.csv')  # Assumes CSV with 'Date' and 'Price' columns
df['Date'] = pd.to_datetime(df['Date'], format='mixed')
df = df.sort_values('Date')
prices = df['Price'].values
dates = df['Date'].values
time_idx = np.arange(len(prices))

In [4]:
# Exploratory Data Analysis (EDA)
# Plot raw price series
plt.figure(figsize=(12, 6))
plt.plot(dates, prices, label='Brent Oil Price (USD)')
plt.title('Brent Oil Prices (1987-2022)')
plt.xlabel('Date')
plt.ylabel('Price (USD/barrel)')
plt.legend()
plt.savefig('eda_prices.png')
plt.close()

In [5]:
# Check stationarity with Augmented Dickey-Fuller test
result = adfuller(prices)
print(f'ADF Statistic: {result[0]}, p-value: {result[1]}')  # p > 0.05 indicates non-stationarity


ADF Statistic: -1.993856011392467, p-value: 0.2892735048934032


In [6]:
# Compute and plot log returns for stationarity
log_returns = np.diff(np.log(prices))
plt.figure(figsize=(12, 6))
plt.plot(dates[1:], log_returns, label='Log Returns')
plt.title('Log Returns of Brent Oil Prices')
plt.xlabel('Date')
plt.ylabel('Log Returns')
plt.legend()
plt.savefig('eda_log_returns.png')
plt.close()

In [7]:
# Analyze volatility (30-day rolling standard deviation)
rolling_std = pd.Series(log_returns).rolling(window=30).std()
plt.figure(figsize=(12, 6))
plt.plot(dates[1:], rolling_std, label='30-Day Rolling Std Dev')
plt.title('Volatility of Brent Oil Log Returns')
plt.xlabel('Date')
plt.ylabel('Standard Deviation')
plt.legend()
plt.savefig('eda_volatility.png')
plt.close()

In [8]:
# Compile event dataset (10-15 major events)
events_data = [
    {'Event_Date': '1991-01-17', 'Event_Description': 'Gulf War Begins'},
    {'Event_Date': '2003-03-20', 'Event_Description': 'Iraq War Begins'},
    {'Event_Date': '2008-09-15', 'Event_Description': 'Global Financial Crisis'},
    {'Event_Date': '2011-02-15', 'Event_Description': 'Arab Spring Onset'},
    {'Event_Date': '2014-06-10', 'Event_Description': 'ISIS Insurgency in Iraq'},
    {'Event_Date': '2014-11-27', 'Event_Description': 'OPEC Maintains Production'},
    {'Event_Date': '2016-11-30', 'Event_Description': 'OPEC Production Cut'},
    {'Event_Date': '2018-05-08', 'Event_Description': 'U.S. Withdraws from Iran Deal'},
    {'Event_Date': '2020-03-08', 'Event_Description': 'OPEC+ Price War'},
    {'Event_Date': '2020-04-12', 'Event_Description': 'OPEC+ Production Cut'},
    {'Event_Date': '2022-02-24', 'Event_Description': 'Russia-Ukraine Conflict Begins'},
]
events = pd.DataFrame(events_data)
events['Event_Date'] = pd.to_datetime(events['Event_Date'])
events.to_csv('events.csv', index=False)

In [10]:
import numpy as np
import pymc as pm
import pytensor
import pytensor.tensor as pt

# Disable C compiler to use Python backend
pytensor.config.cxx = ""
pytensor.config.mode = "FAST_RUN"

# Synthetic data for testing (replace with your actual log_returns and time_idx)
np.random.seed(42)
n = 100
log_returns = np.concatenate([
    np.random.normal(0, 0.1, 30),
    np.random.normal(0.5, 0.1, 30),
    np.random.normal(-0.3, 0.1, 40)
])
time_idx = np.arange(n + 1)  # time_idx should be one element longer than log_returns

with pm.Model() as multi_cp_model:
    n_change_points = 3
    # DiscreteUniform for change points
    tau = pm.DiscreteUniform("tau", lower=0, upper=len(log_returns)-1, shape=n_change_points)
    # Sort tau to ensure ordered change points
    tau_sorted = pm.Deterministic("tau_sorted", pt.sort(tau))
    
    # Priors for means and standard deviation
    mu = pm.Normal("mu", mu=0, sigma=0.1, shape=n_change_points+1)
    sigma = pm.HalfNormal("sigma", sigma=0.1)
    
    # Define piecewise mean using switch
    idx = time_idx[:-1]  # Match length of log_returns
    mu_t = pm.math.switch(idx < tau_sorted[0], mu[0],
                          pm.math.switch(idx < tau_sorted[1], mu[1],
                                         pm.math.switch(idx < tau_sorted[2], mu[2], mu[3])))
    
    # Likelihood
    likelihood = pm.Normal("likelihood", mu=mu_t, sigma=sigma, observed=log_returns)
    
    # MCMC sampling with single core to avoid pickling issues
    trace = pm.sample(2000, tune=1000, return_inferencedata=True, cores=1)

# Optional: Verify model is picklable
import pickle
try:
    pickle.dumps(multi_cp_model)
    print("Model is picklable")
except Exception as e:
    print("Pickling failed:", e)

# Print summary of trace
print(trace.posterior)

Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [tau]
>NUTS: [mu, sigma]


c:\Users\mintesinot\week_10\venv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 747 seconds.
There were 231 divergences after tuning. Increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Pickling failed: 'functools.partial' object has no attribute '__name__'
<xarray.Dataset> Size: 368kB
Dimensions:           (chain: 2, draw: 2000, tau_dim_0: 3, mu_dim_0: 4,
                       tau_sorted_dim_0: 3)
Coordinates:
  * chain             (chain) int64 16B 0 1
  * draw              (draw) int64 16kB 0 1 2 3 4 5 ... 1995 1996 1997 1998 1999
  * tau_dim_0         (tau_dim_0) int64 24B 0 1 2
  * mu_dim_0          (mu_dim_0) int64 32B 0 1 2 3
  * tau_sorted_dim_0  (tau_sorted_dim_0) int64 24B 0 1 2
Data variables:
    tau               (chain, draw, tau_dim_0) int64 96kB 60 60 30 ... 30 60 60
    mu                (chain, draw, mu_dim_0) float64 128kB -0.02669 ... -0.2788
    sigma             (chain, draw) float64 32kB 0.09598 0.09458 ... 0.09148
    tau_sorted        (chain, draw, tau_sorted_dim_0) int64 96kB 30 60 ... 60 60
Attributes:
    created_at:                 2025-08-06T08:04:23.958199+00:00
    arviz_version:              0.22.0
    inference_library:          pymc

In [11]:
# Model diagnostics
print(az.summary(trace, var_names=["tau_sorted", "mu", "sigma"]))
az.plot_trace(trace, var_names=["tau_sorted", "mu", "sigma"])
plt.savefig('model_diagnostics.png')
plt.close()

c:\Users\mintesinot\week_10\venv\Lib\site-packages\arviz\stats\diagnostics.py:596: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
c:\Users\mintesinot\week_10\venv\Lib\site-packages\arviz\stats\diagnostics.py:991: RuntimeWarning: invalid value encountered in scalar divide
  varsd = varvar / evar / 4


                 mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
tau_sorted[0]  30.000  0.000  30.000   30.000      0.000      NaN    4000.0   
tau_sorted[1]  60.000  0.000  60.000   60.000      0.000      NaN    4000.0   
tau_sorted[2]  61.014  3.136  60.000   68.000      1.144    1.722       9.0   
mu[0]          -0.018  0.017  -0.049    0.014      0.001    0.000     858.0   
mu[1]           0.473  0.017   0.440    0.504      0.000    0.000    1434.0   
mu[2]          -0.040  0.131  -0.326    0.148      0.030    0.020      25.0   
mu[3]          -0.296  0.015  -0.325   -0.270      0.001    0.000     831.0   
sigma           0.093  0.007   0.081    0.105      0.000    0.000    1353.0   

               ess_tail  r_hat  
tau_sorted[0]    4000.0    NaN  
tau_sorted[1]    4000.0    NaN  
tau_sorted[2]      12.0   1.17  
mu[0]             538.0   1.00  
mu[1]            3095.0   1.00  
mu[2]              26.0   1.06  
mu[3]            1171.0   1.00  
sigma            2418.0 

In [13]:
# Extract change points
tau_modes = [int(np.bincount(trace.posterior["tau_sorted"].values[:, :, i].flatten()).argmax()) 
             for i in range(n_change_points)]
change_point_dates = [dates[tau + 1] for tau in tau_modes]  # Adjust for log returns offset

# Print change points with proper date formatting
print("Detected Change Points:")
for i, cp_date in enumerate(change_point_dates):
    cp_date = pd.to_datetime(cp_date)  # Convert numpy.datetime64 to Timestamp
    print(f"Change Point {i+1}: {cp_date.strftime('%Y-%m-%d')}")

Detected Change Points:
Change Point 1: 1987-07-03
Change Point 2: 1987-08-14
Change Point 3: 1987-08-14


In [14]:
# Quantify impact
mu_means = trace.posterior["mu"].mean(dim=["chain", "draw"]).values
for i in range(n_change_points):
    price_change_percent = (np.exp(mu_means[i+1]) - np.exp(mu_means[i])) / np.exp(mu_means[i]) * 100
    print(f"Segment {i} to {i+1}: Mean log return from {mu_means[i]:.4f} to {mu_means[i+1]:.4f}, "
          f"Estimated price change: {price_change_percent:.2f}%")

Segment 0 to 1: Mean log return from -0.0175 to 0.4732, Estimated price change: 63.35%
Segment 1 to 2: Mean log return from 0.4732 to -0.0404, Estimated price change: -40.17%
Segment 2 to 3: Mean log return from -0.0404 to -0.2964, Estimated price change: -22.59%


In [16]:
for cp_date in change_point_dates:
    cp_date = pd.to_datetime(cp_date)  # Convert numpy.datetime64 to Timestamp

    relevant_events = events[
        (events['Event_Date'] >= cp_date - window) &
        (events['Event_Date'] <= cp_date + window)
    ]

    if not relevant_events.empty:
        closest_event = relevant_events.iloc[0]
        change_point_events.append({
            'Change_Point_Date': cp_date.strftime('%Y-%m-%d'),
            'Event_Date': pd.to_datetime(closest_event['Event_Date']).strftime('%Y-%m-%d'),
            'Event_Description': closest_event['Event_Description']
        })
    else:
        change_point_events.append({
            'Change_Point_Date': cp_date.strftime('%Y-%m-%d'),
            'Event_Date': 'N/A',
            'Event_Description': 'No event within ±7 days'
        })

In [17]:
# Save change point results
change_points_df = pd.DataFrame(change_point_events)
change_points_df.to_csv('change_points.csv', index=False)

In [19]:
# Visualize price series with change points and events
plt.figure(figsize=(14, 7))
plt.plot(dates, prices, label='Brent Oil Price')

# Plot change points with labels only on the first one
for i, cp_date in enumerate(change_point_dates):
    cp_date = pd.to_datetime(cp_date)  # Convert to pandas Timestamp
    label = f"Change Point: {cp_date.strftime('%Y-%m-%d')}" if i == 0 else ""
    plt.axvline(cp_date, color='r', linestyle='--', label=label)

# Plot events with label only on the first one
for i, event in events.iterrows():
    event_date = pd.to_datetime(event['Event_Date'])
    label = 'Events' if i == 0 else ""
    plt.axvline(event_date, color='g', linestyle=':', alpha=0.5, label=label)

plt.title('Brent Oil Prices with Change Points and Events')
plt.xlabel('Date')
plt.ylabel('Price (USD/barrel)')
plt.legend()
plt.savefig('price_with_change_points.png')
plt.close()
